# Analyse the Results of Running Moran Process Experiment on Different Graphs
This is the newest version of this analysis file, where I can merge the csv of different jobs. 

imports

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from pathlib import Path
from analysis_utils import aggregate_results_2, load_experiment_data, aggregate_results, plot_property_effect, plot_hybrid_density
# change this if on a different computer!
%cd /home/labs/pilpel/matanyaw/moran-process 
from population_graph import GRAPH_PROPS
# Set aesthetic parameters for "publication-quality" plots
sns.set_theme(style="whitegrid", context="notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['lines.linewidth'] = 2.5

batch_name = 'batch_' + 'Test_batch_name_01'

In [ ]:


ROOT = Path(os.getcwd()) 

# Now define your paths relative to ROOT
data_dir = ROOT / "simulation_data"
batch_dir = data_dir / batch_name




In [ ]:
results_df = aggregate_results(batch_dir=batch_dir, delete_temp=False)
print("Results Dataframe Shape : ", results_df.shape)



In [ ]:
results_df = aggregate_results_2(batch_dir=batch_dir, delete_temp=False)
print("Results Dataframe Shape : ", results_df.shape)


In [ ]:
results_df.columns

In [ ]:
# Create a column where steps are NaN if fixation failed
# This allows .agg() to ignore those values automatically for median/std
results_df['steps_success'] = results_df['steps'].where(results_df['fixation'] == True)

analysis_df = results_df.groupby(['wl_hash', 'r', 'graph_name']).agg(
    prob_fixation=('fixation', 'mean'),
    median_steps=('steps_success', 'median'), # NaN values are ignored by default
    mean_steps=('steps_success', 'mean'),
    std_steps=('steps_success', 'std'),
    n_grouped=('fixation', 'size')
).reset_index()

print("Shape before merging: ", analysis_df.shape)
# df_graphs = load_experiment_data('graph_database.csv')       # Graph database
df_graphs = pd.read_csv(os.path.join(batch_dir, 'graph_props.csv'))

# Merge with graph metadata
analysis_df = pd.merge(
    analysis_df, 
    df_graphs, 
    on=['wl_hash', 'graph_name'], 
    how='left', 
    suffixes=('', '_db')
)
# Sorting
analysis_df['z_order'] = (analysis_df['category'] != 'Random').astype(int)
analysis_df = analysis_df.sort_values('z_order').drop(columns='z_order')
analysis_df.to_csv(os.path.join(data_dir, f'{batch_name}_graph_statistics.csv'), index=False)


print("Shape after merging: ", analysis_df.shape)
# Display sample
analysis_df.tail(20)

In [ ]:
len(GRAPH_PROPS)

In [ ]:
df_to_plot = analysis_df[analysis_df['r'] == 1.1]

NEW_GRAPH_PROPS = ['avg_degree', 'max_degree']
print(GRAPH_PROPS)
plot_property_effect(df_to_plot, 'median_steps', 'std_steps')

plot_property_effect(analysis_df, 'median_steps', 'prob_fixation')

# --- EXAMPLES OF USAGE ---
for prop in GRAPH_PROPS:
    # plot_property_effect(df_to_plot, prop, 'median_steps')
    plot_hybrid_density(df_to_plot, prop, 'median_steps', density_threshold=50, with_violin=True)
    plot_hybrid_density(df_to_plot, prop, 'prob_fixation', density_threshold=50, with_violin=True)
